In [28]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble   import RandomForestClassifier
from sklearn.svm import NuSVC

In [29]:
pd.set_option('display.max_columns',         500)
pd.set_option('display.width',       1000)
pd.set_option('display.max_colwidth',        -1)

In [30]:
df = pd.read_csv(
    filepath_or_buffer='..\\data\\covtype.data',
    sep=',',
    header=None
)

MemoryError: 

In [ ]:
df.head()

In [ ]:
columns_map = {

    # 10 quantitative variables

    0: 'elevation',
    1: 'aspect',
    2: 'slope',
    3: 'hor_dist_to_hydr',
    4: 'vert_dist_to_hydr',
    5: 'hor_dist_to_roadway',
    6: 'hillshade_9am',
    7: 'hillshade_noon',
    8: 'hillshade_3pm',
    9: 'hor_dist_to_ignition',

    # 4 binary wilderness areas

    10: 'rawah',
    11: 'neota',
    12: 'comanche',
    13: 'cache_la_poudre',

    # Soil Types: 1 to 40 ; based on the USFS Ecological Landtype Units (ELUs) for this study area

    14: '2702',
    15: '2703',
    16: '2704',
    17: '2705',
    18: '2706',
    19: '2717',
    20: '3501',
    21: '3502',
    22: '4201',
    23: '4703',
    24: '4704',
    25: '4744',
    26: '4758',
    27: '5101',
    28: '5151',
    29: '6101',
    30: '6102',
    31: '6731',
    32: '7101',
    33: '7102',
    34: '7103',
    35: '7201',
    36: '7202',
    37: '7700',
    38: '7701',
    39: '7702',
    40: '7709',
    41: '7710',
    42: '7745',
    43: '7746',
    44: '7755',
    45: '7756',
    46: '7757',
    47: '7790',
    48: '8703',
    49: '8707',
    50: '8708',
    51: '8771',
    52: '8772',
    53: '8776',

    # Forest Cover Type Classes

    54: 'cover_type'

}

In [ ]:
df.rename(
    columns=columns_map,
    inplace=True
)

In [ ]:
df.head()

In [ ]:
# euclidean distance calculation  
df['dist_to_hydr'] = np.sqrt( df.hor_dist_to_hydr**2 + df.vert_dist_to_hydr**2)

In [ ]:
df.drop(['hor_dist_to_hydr','vert_dist_to_hydr'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df['hillshade'] = np.sqrt(  df['hillshade_9am']**2 + df['hillshade_noon']**2 + df['hillshade_3pm']**2)

In [33]:
df.drop(['hillshade_9am','hillshade_noon','hillshade_3pm'], axis=1, inplace=True)

MemoryError: 

In [ ]:
df.to_pickle('..\\data\\OHE.pkl')

In [32]:
df['wilderness_area'] = df[['rawah', 'neota',
                            'comanche', 'cache_la_poudre']].idxmax(1)
df.drop(['rawah', 'neota', 'comanche', 'cache_la_poudre'], axis=1, inplace=True)

MemoryError: 

In [ ]:
soil = ['2702', '2703', '2704', '2705', '2706', '2717', '3501', '3502', '4201', '4703',
        '4704', '4744', '4758', '5101', '5151', '6101', '6102', '6731', '7101', '7102',
        '7103', '7201', '7202', '7700', '7701', '7702', '7709', '7710', '7745', '7746',
        '7755', '7756', '7757', '7790', '8703', '8707', '8708', '8771', '8772', '8776']
df['soil']= df[soil].idxmax(1)
df['geo_soil'] = df['soil'].astype(str).str[0]
df['climate_soil'] = df['soil'].astype(str).str[1]
df.drop(['soil'], axis=1, inplace=True)

In [ ]:
df.to_pickle('..\\data\\no_OHE.pkl')